In [2]:
from pydantic import BaseModel, EmailStr, BeforeValidator, Field
from datetime import datetime
from typing import Optional, Annotated

PyObjectId = Annotated[str, BeforeValidator(str)]

class User(BaseModel):
    uid: str
    first_name: str
    last_name: str
    email: str
    password_hash: str
    admin: bool

In [3]:
from passlib.context import CryptContext
bcrypt_context = CryptContext(schemes=['bcrypt'], deprecated='auto')

def get_password_hash(password: str) -> str:
    return bcrypt_context.hash(password)

In [9]:
import os
from dotenv import load_dotenv
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

# get password and uri for database connection
load_dotenv()
db_password = os.getenv('MONGO_PASSWORD')
uri = os.getenv('DATABASE_URI').replace('db_password', db_password)

# create client and connect to server
client = MongoClient(uri, server_api=ServerApi('1'))

# ping for confirmation and update databases
db = client['bigcat']

In [10]:
import requests
import random
from uuid import uuid4

response = requests.get('https://dummyjson.com/users')
data = response.json()['users']

email_suffixes = ['gmail.com', 'outlook.com', 'icloud.com', 'yahoo.com']

for d in data:
    # change email to something real

    random_number = random.randint(0, len(email_suffixes)-1)
    email = d['email'].replace('x.dummyjson.com', email_suffixes[random_number])

    user = User(
        uid=str(uuid4()),
        first_name=d['firstName'],
        last_name=d['lastName'],
        email=email,
        password_hash=get_password_hash(d['password']),
        admin=False
    )

    db['user'].insert_one(user.__dict__)

print('Done!')

Done!


In [ ]:
import json

with open('seating_chart.json', 'r') as file:
    data = json.load(file)

del data['schema']
for key, val in data.items():
    print(key, val)

3TEN Austin City Limits Live {'layout': {'GA': [1, 0, 350, 50]}}
Antone's Austin {'layout': {'GA': [1, 0, 750, 15]}}
Austin City Limits Live at The Moody Theater {'layout': {'GA': [1, 0, 300, 35, 50], 'MEZZ': [8, 3, 25, 45], 'BALC': [6, 12, 22, 30]}}
Brauntex Performing Arts Theatre {'layout': {'GA': [1, 0, 586, 40]}}
Cheatham Street Warehouse {'layout': {'GA': [1, 0, 500, 20]}}
Empire - Control Room {'layout': {'GA': [1, 0, 300, 35]}}
Germania Insurance Amphitheater {'layout': {'GA-Pit': [1, 0, 800, 50], '100-Level': [7, 12, 30, 65], '200-Level': [4, 31, 30, 55], 'GA-Lawn': [1, 0, 7000, 20]}}
Gruene Hall {'layout': {'GA': [1, 0, 800, 20]}}
Long Center for the Performing Arts - Dell Hall {'layout': {'Orchestra': [3, 17, 20, 45], 'Parterre': [3, 9, 25, 25]}}
Moody Amphitheater {'layout': {'GA': [1, 0, 5000, 40]}}
Moody Center {'layout': {'Pit': [2, 0, 200, 200], 'GA': [9, 0, 100, 100], '100-Level': [14, 10, 30, 90], '200-Level': [9, 7, 30, 70], 'S-Level': [9, 3, 7, 105], 'P-Level': [9, 

In [17]:
names_list = ['three_ten', 'antones', 'acl_live', 'brantex', 'cheatham', 'empire_control_room', 'germania', 'gruene', 'long_center', 'moody', 'paramount', 'scoot', 'stubbs', 'concourse_project', 'zilker']

for name in names_list:
    user = User(
            uid=str(uuid4()),
            first_name='Admin',
            last_name=name.title().replace('_', ' '),
            email=name + '@bigcat.com',
            password_hash=get_password_hash(name + '_admin'),
            admin=True
        )

    db['user'].insert_one(user.__dict__)

In [ ]:
pipeline = [
    {
        '$group': {
            '_id': {'last_name': '$last_name',},
            'duplicate_ids': {'$push': '$uid'},
            'count': {'$sum': 1}
        }
    },
    {
        '$match': {
            'count': {'$gt': 1}
        }
    }
]

duplicates = list(db['user'].aggregate(pipeline))
print(f'{len(duplicates)} duplicates found.')

counter = 0

for doc in duplicates:
        ids_to_remove = doc['duplicate_ids'][1:]  # Keep the first, remove the rest
        if ids_to_remove:
            db['user'].delete_many({'uid': {'$in': ids_to_remove}})
        counter += 1

print(f"{counter} duplicates documents removed.")

2 duplicates found.
2 duplicate documents removed.
